In [1]:
from book_model import BookAllField

## 2.5.7. Aggregation(聚合)

MongoDB provides some aggregation methods out of the box, but there are not as many as you typically get with an RDBMS. MongoEngine provides a wrapper around the built-in methods and provides some of its own, which are implemented as Javascript code that is executed on the database server.

MongoDB提供聚合方法;同时，MongoEngine(具体由 `mongoengine.queryset.QuerySet`提供)提供了这些方法中常用方法的快捷使用方法。

### 2.5.7.1. Counting results

Just as with limiting and skipping results, there is a method on a QuerySet object – `count()`:
```python
num_users = User.objects.count()
```

In [2]:
BookAllField.objects.count()

20004

In [3]:
BookAllField.objects(name="大学").count()

13

### 2.5.7.2. Further aggregation(更强大的聚合)

#### **sum**  
You may sum over the values of a specific field on documents using sum():
```python
yearly_expense = Employee.objects.sum('salary')
```
> If the field isn’t present on a document, that document will be ignored from the sum.  
> 字段在文档中不存在，在数据加总时会被忽略

In [5]:
# 查询书 “大学”， 所有单价的总和
BookAllField.objects(name="大学").sum("price")

1329.51

#### **average()**  
To get the average (mean) of a field on a collection of documents, use `average()`:

```python
mean_age = User.objects.average('age')
```

In [6]:
# 查询书 “大学”， 平均单价
BookAllField.objects(name="大学").average("price")

102.27

In [7]:
# 查询书 “大学”， 平均单价
BookAllField.objects(name="大学").max("price")
# max报错，目前只能使用 sum和average,其余需要直接使用 数据库提供的聚合查询方法

AttributeError: 'QuerySet' object has no attribute 'max'

#### **distinct**, 输出列表

In [12]:
len(BookAllField.objects().distinct("name"))

13409

In [20]:
BookAllField.objects().distinct("ifnewbook")

[False, True]

#### **item_frequencies**, 字段值的统计频率字典

As MongoDB provides native lists, MongoEngine provides a helper method to get a dictionary of the frequencies of items in lists across an entire collection – `item_frequencies()`. An example of its use would be generating `“tag-clouds”`:

```python
class Article(Document):
    tag = ListField(StringField())

# After adding some tagged articles...
tag_freqs = Article.objects.item_frequencies('tag', normalize=True)

from operator import itemgetter
top_tags = sorted(tag_freqs.items(), key=itemgetter(1), reverse=True)[:10]
```

In [19]:
# 注意， LazyReference,字典和嵌入式文档不能使用频率统计；列表的频率统计会以列表中的值作为基础；引用类型会显示 id
BookAllField.objects(name="大学").item_frequencies("tag")

{'必读': 5,
 '一直': 1,
 '软件': 1,
 '主题': 1,
 '历史': 4,
 '经典': 7,
 '电子': 1,
 '东西': 1,
 '提供': 1,
 '位置': 1,
 '电影': 2}

### 2.5.7.3. MongoDB aggregation API

If you need to run aggregation pipelines, MongoEngine provides an entry point to *Pymongo’s aggregation framework* through `aggregate()`. **Check out Pymongo’s documentation** for the syntax and pipeline. An example of its use would be:

```python
class Person(Document):
    name = StringField()

Person(name='John').save()
Person(name='Bob').save()

pipeline = [
    {"$sort" : {"name" : -1}},
    {"$project": {"_id": 0, "name": {"$toUpper": "$name"}}}
    ]
data = Person.objects().aggregate(pipeline)
assert data == [{'name': 'BOB'}, {'name': 'JOHN'}]
```